# Web Scraping: Selenium

A menudo, los datos están disponibles públicamente para nosotros, pero no en una forma que sea fácilmente utilizable. Ahí es donde entra en juego el web scraping, podemos usar web scraping para obtener nuestros datos deseados en un formato conveniente que luego se puede usar. a continuación, mostraré cómo se puede extraer información de interés de un sitio web usando el paquete Selenium en Python. Selenium nos permite manejar una ventana del navegador e interactuar con el sitio web mediante programación. 

Selenium también tiene varios métodos que facilitan la extracción de datos.
En este Jupyter Notebook vamos a usar Python 3 en Windows.

En primer lugar, tendremos que descargar un controlador.

Usaremos ChromeDriver para Google Chrome. Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/). Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su versión actual de Google Chrome.

Como saber cual es la version de chrome que utilizo simple utilizamos pegamos el siguiente enlace en la barra de chrome chrome://settings/help

Antes de comenzar se preguntaran si ya se BeautifulSoup cual es la diferencia con Selenium.

A diferencia BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM



## Comencemos importando las bibliotecas que usaremos:

In [ ]:
from selenium import webdriver
import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd

El objeto driver es con el que trabajaremos a partir de ahora

In [ ]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'chromedriver.exe'
options  = webdriver.ChromeOptions()

In [ ]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [ ]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://insolvencyinsider.ca/filing/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

Ahora si queremos hacer click en el boton de "Load more"..

Selenium proporciona varios métodos para localizar elementos en la página web. Usaremos el método find_element_by_xpath() para crear un objeto de botón, con el que luego podremos interactuar:

In [ ]:
loadMore = driver.find_element_by_xpath(xpath="")

Antes de continuar, necesitaremos saber cuántas páginas hay para saber cuántas veces debemos hacer clic en el botón. Necesitaremos una forma de extraer el código fuente del sitio web. Afortunadamente, este proceso es relativamente sencillo con las bibliotecas urllib3 y re.

In [ ]:
url = "https://insolvencyinsider.ca/filing/"
http = urllib3.PoolManager()
r = http.request("GET", url)
text = str(r.data)


```text``` ahora es una cadena. Ahora, necesitamos una forma de extraer total_pages de nuestra cadena de texto. Imprima texto para ver cómo podemos extraerlo usando RegEx con el paquete re. Podemos totalizar_páginas así:

In [ ]:
totalPagesObj = re.search(pattern='"total_pages":\d+', string=text)
totalPagesStr = totalPagesObj.group(0)
totalPages = int((re.search(pattern="\d+", string=totalPagesStr)).group(0))

El método de búsqueda toma un patrón y una cadena. En este caso nuestro patrón es '"total_pages":\d+' . Si no está familiarizado con RegEx, todo esto significa que estamos buscando la cadena "total_pages": con dos o más dígitos después de los dos puntos. \d se refiere a un dígito entre 0 y 9, mientras que + indica que Python debe buscar una o más de las expresiones regulares anteriores. Puedes leer más sobre el paquete re aquí. El método search() devuelve un objeto Match. re proporciona el método group() que devuelve uno o más subgrupos de la coincidencia. Pasamos 0 como argumento para indicar que queremos el parche completo. La tercera línea simplemente extrae el entero correspondiente a total_pages de la cadena.

In [ ]:
print(totalPagesObj)
print(totalPagesStr)
print(totalPages)

Con eso completo, ahora podemos cargar todas las páginas de Insolvency Insider. Podemos hacer clic en el botón Cargar más accediendo al método click() del objeto. Esperamos tres segundos entre clics para no sobrecargar el sitio web.

Recuerde que el total de las páginas son 88 pero comenzamos en 0 asi que es 88-1

In [ ]:
for i in range(totalPages-1):
    loadMore.click()
    time.sleep(3)

Una vez que ejecute esto, debería ver que se hace clic en el botón Cargar más y que se cargan las páginas restantes.
Una vez que se carga cada página, podemos comenzar a raspar el contenido. Ahora, eliminar ciertos elementos como el nombre de presentación, la fecha y la hiperreferencia es bastante sencillo. Podemos usar los métodos find_elements_by_class_name() y find_elements_by_xpath() de Selenium (importante la ```s``` extra después de element):

In [ ]:
filingNamesElements = driver.find_elements_by_class_name("")

filingDateElements = driver.find_elements_by_class_name("")

filingHrefElements = driver.find_elements_by_xpath("")

También nos gustaría conocer los metadatos de presentación, es decir, el tipo de archivo, el sector de la empresa y el lugar en la que operan. Extraer estos datos requiere un poco más de trabajo.

In [ ]:
filingMetas = []
for i in range(len(filingNamesElements) + 1):
    filingMetai = driver.find_elements_by_xpath(("" %(i)))
    for element in filingMetai:
        filingMetaTexti = element.text
        filingMetas.append(filingMetaTexti)

De cada elemento de la presentación de Metas podemos extraer el tipo de presentación, la industria y la provincia, así:

In [ ]:
metaDict = {"Filing Type": [], "Industry": [], "Province": []}
for filing in filingMetas:
    filingSplit = filing.split("\n")
  
    for item in filingSplit:
        itemSplit = item.split(":")

        
        if itemSplit[0] == "Filing Type":
            metaDict["Filing Type"].append(itemSplit[1])
        elif itemSplit[0] == "Industry":
            metaDict["Industry"].append(itemSplit[1])
        elif itemSplit[0] == "Province":
            metaDict["Province"].append(itemSplit[1])
            
    if "Filing Type" not in filing:
        metaDict["Filing Type"].append("NA")
    elif "Industry" not in filing:
        metaDict["Industry"].append("NA")
    elif "Province" not in filing:
        metaDict["Province"].append("NA")

In [ ]:
for key in metaDict:
    print(len(metaDict[key]))

Ahora, todavía tenemos que poner nuestros nombres y fechas de presentación en las listas. Hacemos esto agregando el texto de cada elemento a una lista usando el método text() de antes:

In [ ]:
filingName = []
filingDate = []
filingLink = []
# para cada elemento en la lista de elementos de nombre de archivo, agrega el
# texto del elemento a la lista de nombres de archivo.
for element in filingNamesElements:
    filingName.append(element.text)
# para cada elemento en la lista de elementos de la fecha de presentación, agrega el
# texto del elemento a la lista de fechas de presentación.
for element in filingDateElements:
    filingDate.append(element.text)
for link in filingHrefElements:
    if link.get_attribute("href"):
        filingLink.append(link.get_attribute("href"))

Una vez que tengamos eso, estamos listos para poner todo en un diccionario y luego crear un DataFrame de pandas:

In [ ]:
# Crea un diccionario final con nombres y fechas de archivo.
fullDict = {
    "Filing Name": filingName,
    "Filing Date": filingDate, 
    "Filing Type": metaDict["Filing Type"],
    "Industry": metaDict["Industry"],
    "Province": metaDict["Province"],
    "Link": filingLink
}
# Crea un DataFrame.
df = pd.DataFrame(fullDict)
df["Filing Date"] = pd.to_datetime(df["Filing Date"], infer_datetime_format=True)

In [ ]:
df

------------------------

# Ahora algo más visual

In [ ]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [ ]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

La página de Filmaffinity se ha abierto

Pero....

Nos hemos encontrado con un pop-up que nos pide aceptar cookies

1. Buscamos el botón
2. Hacemos click en el botón

Vamos a quitar el boton para seguir

In [ ]:
elements_by_tag = driver.find_elements_by_tag_name('button')
elements_by_class_name = driver.find_elements_by_class_name('css-v43ltw')
element_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]')

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [ ]:
dir(element_by_xpath)
# obtenemos todos sus métodos y atributos:

Podemos evaluar que tipo de elemento es (tag)

In [ ]:
element_by_xpath.tag_name

Podemos sacar el valor que tiene (el texto)

In [ ]:
element_by_xpath.text

In [ ]:
for i in range(0,len(elements_by_tag)):
    print(elements_by_tag[i].text)

Incluso podemos guardar una imagen del elemento

In [ ]:
type(element_by_xpath)
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

In [ ]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot('mi_imagen.png')

Evaluamos que elementos hemos encontrado por el tag:

In [ ]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento',index, 'es', element.text)
    print('El tag del elemento',index, 'es', element.tag_name)
    element.screenshot('mi_imagen'+str(index)+'.png')

Basta de tonterias seguimos

Instanciamos el elemento del tag [2] en la variable boton aceptar

In [ ]:
boton_aceptar = elements_by_tag[2]

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [ ]:
boton_aceptar.click()

Buscamos una película por título

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
buscador = driver.find_element_by_xpath('')

In [ ]:
buscador.send_keys('')

In [ ]:
buscador.clear()

In [ ]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [ ]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [ ]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

In [ ]:
mis_secciones = menu_lateral.find_elements_by_tag_name('a')

2. Vemos con cuál nos tenemos que quedar

In [ ]:
for a in mis_secciones:
    if a.text == 'Próximos estrenos':
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [ ]:
cajon_central = driver.find_elements_by_id('main-wrapper-rdcat')

In [ ]:
type(cajon_central)

In [ ]:
for semana in cajon_central:
    print(semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attribute('id'))

In [ ]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2022-02-25':
        break

Buscamos cómo acceder a las películas

In [ ]:
caratulas = semana.find_elements_by_class_name('')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name('a').get_attribute('href'))

In [ ]:
lista_pelis

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [ ]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [ ]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [ ]:
titulo

2. Creamos una lista a partir de la ficha técnica

In [ ]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_names = []
ficha_values = []

for name in ficha.find_elements_by_tag_name('dt'):
    ficha_names.append(name.text)
for value in ficha.find_elements_by_tag_name('dd'):
    ficha_values.append(value.text)

In [ ]:
ficha_values

3. Creamos un dataframe con la info

In [ ]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_names)
len(columns)

In [ ]:
values = [titulo, nota, votos]
values.extend(ficha_values)
len(values)

In [ ]:
pd.DataFrame([values],columns=columns)

Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [ ]:
def sacar_info(driver):
    
    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    try:
        nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]').text
        votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')
    
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    
    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements_by_tag_name('dt'):
        ficha_names.append(name.text)
    for value in ficha.find_elements_by_tag_name('dd'):
        ficha_values.append(value.text)
        
    return ficha_names, ficha_values

def montar_df(ficha_names, ficha_values, titulo, nota, votos):
    
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_names)
    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns = columns)
    
def nueva_pelicula(driver):
    
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = sacar_ficha(ficha)
    df_peli = montar_df(ficha_names, ficha_values, titulo, nota, votos)
    
    return df_peli


Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [ ]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [ ]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [ ]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [ ]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [ ]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

In [ ]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_peli = nueva_pelicula(driver)
    df_peliculas = df_peliculas.append(nueva_peli)

In [ ]:
df_peliculas.info()

In [ ]:
df_peliculas

¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes